In [1]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.window import Window
from sklearn import preprocessing # https://github.com/Snowflake-Labs/snowpark-python-demos/tree/main/sp4py_utilities
from snowflake.snowpark.functions import col
import snowflake
import snowflake.snowpark

import getpass
import pandas as pd
import matplotlib.pyplot as plt
import decimal
import joblib 
from datetime import datetime, timedelta
import random
import math

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
accountname = "HIIOYKL-IX77996" # ORGNAME-ACCOUNTNAME (separated by minus sign)
username =  "Nathan" # SNOWFLAKE-USERNAME
password = "Nathan5!"# SNOWFLAKE-PASSWORD

In [3]:
connection_parameters = {
    "account": accountname,
    "user": username,
    "password": password,
    "role": "ACCOUNTADMIN",
    "database": "FROSTBYTE_TASTY_BYTES",

    "warehouse": "COMPUTE_WH"
}

session = Session.builder.configs(connection_parameters).create()

### Generating Usual Route

In [5]:
truck_df=pd.read_csv("truck_details.csv")

In [6]:
truck_df[""]

,TRUCK_ID,STARTING_TIME,ENDING_TIME
0,17,8,23
1,27,8,22
2,21,8,22
3,28,8,22
4,47,8,22
5,46,8,22
6,33,8,22
7,34,8,22
8,43,8,22
9,35,8,22


In [12]:
def calculate_list_and_mod(row):
    working_hours = row['working_hour']
    num_locs = row['Num_of_locs']
    each_loc_hours = working_hours // num_locs
    remainder = working_hours % num_locs
    result_list = [each_loc_hours] * num_locs
    for i in range(remainder):
        result_list[i] += 1
    return result_list
def calculate_start_time(row):
    result_list=row["shift_hours"]
    num_locs = row['Num_of_locs']
    start_times = [row['STARTING_TIME']]
    for i in range(1, num_locs):
        start_times.append(start_times[-1] + result_list[i - 1])

    start_times.append(row["ENDING_TIME"])
    return start_times
truck_df['Num_of_locs']=2
truck_df["working_hour"]=truck_df['ENDING_TIME']-truck_df['STARTING_TIME']+1
truck_df["shift_hours"]=truck_df.apply(calculate_list_and_mod, axis=1)
truck_df["start_time"]=truck_df.apply(calculate_start_time, axis=1)

In [13]:
truck_df

,TRUCK_ID,STARTING_TIME,ENDING_TIME,working_hour,shift_hours,Num_of_locs,start_time
0,17,8,23,16,"[8, 8]",2,"[8, 16, 23]"
1,27,8,22,15,"[8, 7]",2,"[8, 16, 22]"
2,21,8,22,15,"[8, 7]",2,"[8, 16, 22]"
3,28,8,22,15,"[8, 7]",2,"[8, 16, 22]"
4,47,8,22,15,"[8, 7]",2,"[8, 16, 22]"
5,46,8,22,15,"[8, 7]",2,"[8, 16, 22]"
6,33,8,22,15,"[8, 7]",2,"[8, 16, 22]"
7,34,8,22,15,"[8, 7]",2,"[8, 16, 22]"
8,43,8,22,15,"[8, 7]",2,"[8, 16, 22]"
9,35,8,22,15,"[8, 7]",2,"[8, 16, 22]"


In [18]:
algo_table = session.table('ROUTING."ALGO_DATA(With Year)"')
algo_df = algo_table.collect()
algo_df=pd.DataFrame(algo_df,columns=algo_table.columns)
df_drop=algo_df.drop("YEAR",axis=1)

In [ ]:
algo_table.groupby(["Hour","LOCATION_ID"]).max()["SUM(ORDER_TOTAL)"]

In [20]:
df_drop

,TRUCK_ID,MONTH,HOUR,DOW,DAY,PUBLIC_HOLIDAY,"""SUM(ORDER_TOTAL)""",LAT,LONG,LOCATION_ID,...,CITY_SEATTLE_ENCODED,CITY_DENVER_ENCODED,"""CITY_San Mateo_encoded""","""CITY_New York City_encoded""",CITY_BOSTON_ENCODED,REGION_NY_ENCODED,REGION_MA_ENCODED,REGION_CO_ENCODED,REGION_WA_ENCODED,REGION_CA_ENCODED
0,17,9,16,4,10,0,1305.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
1,17,9,17,4,10,0,610.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
2,17,9,18,4,10,0,3461.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
3,17,9,19,4,10,0,5221.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
4,17,9,20,4,10,0,8929.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540196,21,6,18,4,4,0,3517.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
540197,21,6,19,4,4,0,2466.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
540198,21,6,20,4,4,0,2297.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
540199,21,6,21,4,4,0,2289.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0


In [46]:
route_df=df_drop.groupby(["TRUCK_ID","HOUR","LOCATION_ID"]).max()['"SUM(ORDER_TOTAL)"'].reset_index()

In [47]:
route_df

,TRUCK_ID,HOUR,LOCATION_ID,"""SUM(ORDER_TOTAL)"""
0,1,8,1059,457.5
1,1,8,1061,74.0
2,1,8,1062,577.5
3,1,8,1063,332.0
4,1,8,1065,662.0
...,...,...,...,...
281656,75,22,15219,10709.0
281657,75,22,15220,16588.0
281658,75,22,15474,17118.0
281659,75,22,15513,17550.0


In [32]:
truck_df

,TRUCK_ID,STARTING_TIME,ENDING_TIME,working_hour,shift_hours,Num_of_locs,start_time
0,17,8,23,16,"[8, 8]",2,"[8, 16, 23]"
1,27,8,22,15,"[8, 7]",2,"[8, 16, 22]"
2,21,8,22,15,"[8, 7]",2,"[8, 16, 22]"
3,28,8,22,15,"[8, 7]",2,"[8, 16, 22]"
4,47,8,22,15,"[8, 7]",2,"[8, 16, 22]"
5,46,8,22,15,"[8, 7]",2,"[8, 16, 22]"
6,33,8,22,15,"[8, 7]",2,"[8, 16, 22]"
7,34,8,22,15,"[8, 7]",2,"[8, 16, 22]"
8,43,8,22,15,"[8, 7]",2,"[8, 16, 22]"
9,35,8,22,15,"[8, 7]",2,"[8, 16, 22]"


In [64]:
truck_df["location_visited"]=pd.Series([[]] * len(truck_df))
truck_df["predicted_earning"]=pd.Series([[]] * len(truck_df))

In [54]:
def calculate_next_start_time(start_times, current_hour):
    if current_hour not in start_times:
        return None
    index = start_times.index(current_hour) + 1
    if index == len(start_times):
        return None
    return start_times[index]
 

In [55]:
truck_df

,TRUCK_ID,STARTING_TIME,ENDING_TIME,working_hour,shift_hours,Num_of_locs,start_time,location_visited,predicted_earning
0,17,8,23,16,"[8, 8]",2,"[8, 16, 23]",[],[]
1,27,8,22,15,"[8, 7]",2,"[8, 16, 22]",[],[]
2,21,8,22,15,"[8, 7]",2,"[8, 16, 22]",[],[]
3,28,8,22,15,"[8, 7]",2,"[8, 16, 22]",[],[]
4,47,8,22,15,"[8, 7]",2,"[8, 16, 22]",[],[]
5,46,8,22,15,"[8, 7]",2,"[8, 16, 22]",[],[]
6,33,8,22,15,"[8, 7]",2,"[8, 16, 22]",[],[]
7,34,8,22,15,"[8, 7]",2,"[8, 16, 22]",[],[]
8,43,8,22,15,"[8, 7]",2,"[8, 16, 22]",[],[]
9,35,8,22,15,"[8, 7]",2,"[8, 16, 22]",[],[]


In [66]:
# Iterate over each hour
for hour in range(24):
    
    trucks_starting_now = truck_df[truck_df['start_time'].apply(lambda start_times: hour in start_times)]
    trucks_starting_now ['next_start_time'] =  trucks_starting_now ['start_time'].apply(lambda start_times: calculate_next_start_time(start_times,hour))
    
    
    for index, truck in  trucks_starting_now .iterrows():
         
         
         ### ditance calculator
         ## remove values from available_locatio based on distance
         ##if u wan any other way also can
        new_list=[]

        if pd.notna(truck["next_start_time"]):
       
            route_df_h=route_df[(route_df["HOUR"]>=hour) & (route_df["HOUR"]<=truck["start_time"][truck["start_time"].index(hour)+1] )] 
            
            route_df_t= route_df_h[ route_df_h["TRUCK_ID"]==truck["TRUCK_ID"]]
    
                    
            route_df_grpby=route_df_t.groupby("LOCATION_ID").sum()['"SUM(ORDER_TOTAL)"']
            if len(truck["location_visited"])>0:
                route_df_grpby.drop(truck["location_visited"][0])
            
            location =  route_df_grpby.idxmax()
        
        
            
      
            truck_df.at[index,'location_visited'] =   list(truck_df.at[index,'location_visited']) + [location]
            truck_df.at[index,'predicted_earning'] = list(truck_df.at[index,'predicted_earning']) + [route_df_grpby.max()]
                

            

  

C:\Users\vibu\AppData\Local\Temp\ipykernel_8888\529552879.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trucks_starting_now ['next_start_time'] =  trucks_starting_now ['start_time'].apply(lambda start_times: calculate_next_start_time(start_times,hour))
C:\Users\vibu\AppData\Local\Temp\ipykernel_8888\529552879.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trucks_starting_now ['next_start_time'] =  trucks_starting_now ['start_time'].apply(lambda start_times: calculate_next_start_time(start_times,h

In [68]:
def haversine_distance(lat1, lon1, lat2, lon2):
        # Convert latitude and longitude from degrees to radians
        lat1_rad = math.radians(lat1)
        lon1_rad = math.radians(lon1)
        lat2_rad = math.radians(lat2)
        lon2_rad = math.radians(lon2)

        # Haversine formula
        dlon = lon2_rad - lon1_rad
        dlat = lat2_rad - lat1_rad
        a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        distance = 6371 * c  # Radius of the Earth in kilometers
        return distance

In [71]:
def get_lat_long(location_id):
    row = algo_df[algo_df['LOCATION_ID'] == location_id]
    if not row.empty:
        return row['LAT'].values[0], row['LONG'].values[0]
    return None, None

# Calculate the total distance traveled for each truck
total_distances = []
for _, row in truck_df.iterrows():
    starting_location = row['location_visited'][0]
    location_visited = row['location_visited'][1:]
    
    total_distance = 0
    prev_lat, prev_lon = get_lat_long(starting_location)  # Get the latitude and longitude of the starting location
    
    # Loop through each location in location_visited
    for location_id in location_visited:
        lat, lon = get_lat_long(location_id)
        
        # If latitude and longitude are found, calculate the distance between the two locations
        if lat is not None and lon is not None:
            total_distance += haversine_distance(prev_lat, prev_lon, lat, lon)
        
            # Update the previous location
            prev_lat, prev_lon = lat, lon
    
    total_distances.append(total_distance)

# Add the total_distances list as a new column to the truck_df
truck_df['total_distance_traveled'] = total_distances

In [76]:
truck_df.to_csv("truck_usual_routine.csv",index=False,header=True)

## Generating Data

In [47]:
truck_cols=['TRUCK_ID','MENU_TYPE_GYROS_ENCODED', 'MENU_TYPE_CREPES_ENCODED',
       'MENU_TYPE_BBQ_ENCODED', 'MENU_TYPE_SANDWICHES_ENCODED',
       'MENU_TYPE_Mac & Cheese_encoded', 'MENU_TYPE_POUTINE_ENCODED',
       'MENU_TYPE_ETHIOPIAN_ENCODED', 'MENU_TYPE_TACOS_ENCODED',
       'MENU_TYPE_Ice Cream_encoded', 'MENU_TYPE_Hot Dogs_encoded',
       'MENU_TYPE_CHINESE_ENCODED', 'MENU_TYPE_Grilled Cheese_encoded',
       'MENU_TYPE_VEGETARIAN_ENCODED', 'MENU_TYPE_INDIAN_ENCODED',
       'MENU_TYPE_RAMEN_ENCODED']
location_cols=[ 'CITY_SEATTLE_ENCODED',
       'CITY_DENVER_ENCODED', 'CITY_San Mateo_encoded',
       'CITY_New York City_encoded', 'CITY_BOSTON_ENCODED',
       'REGION_NY_ENCODED', 'REGION_MA_ENCODED', 'REGION_CO_ENCODED',
       'REGION_WA_ENCODED', 'REGION_CA_ENCODED', 'LAT', 'LONG', 'LOCATION_ID']

In [48]:
date = '2021-8-23'
datetime_object = datetime.strptime(date, '%Y-%m-%d')

In [49]:
def generate_date_data(df,datetime_object):
    df['date'] = pd.to_datetime(datetime_object)
    df['MONTH'] = df['date'].dt.month
    df['DOW'] = df['date'].dt.weekday
    df['DAY'] = df['date'].dt.day
    df['WOM'] = (df['DAY'] - 1) // 7 + 1
    df['YEAR'] = df['date'].dt.year

    public_holidays = [
     {'Month': 7, 'Day': 4, 'DOW': None, 'WOM': None},  # 4th of July
   {'Month': 12, 'Day': 24, 'DOW': None, 'WOM': None},  # Christmas Eve
    {'Month': 12, 'Day': 25, 'DOW': None, 'WOM': None},  # Christmas Day
    {'Month': 10, 'Day': None, 'DOW': '0', 'WOM': 2},  # Columbus Day (second Monday in October)
    {'Month': 6, 'Day': 19, 'DOW': None, 'WOM': None},  # Juneteenth
    {'Month': 9, 'Day': None, 'DOW': '0', 'WOM': 1},  # Labor Day (first Monday in September)
    {'Month': 1, 'Day': None, 'DOW': '0', 'WOM': 3},  # Martin Luther King, Jr. Day (third Monday in January)
    {'Month': 5, 'Day': None, 'DOW': '0', 'WOM': -1},  # Memorial Day (last Monday in May)
    {'Month': 1, 'Day': 1, 'DOW': None, 'WOM': None},  # New Year's Day
    {'Month': 12, 'Day': 31, 'DOW': None, 'WOM': None},  # New Year's Eve
    {'Month': 11, 'Day': None, 'DOW': '3', 'WOM': 4},  # Thanksgiving Day (fourth Thursday in November)
    {'Month': 11, 'Day': None, 'DOW': '2', 'WOM': 4},  # Thanksgiving Eve (fourth Wednesday in November)
    {'Month': 2, 'Day': 14, 'DOW': None, 'WOM': None},  # Valentine's Day
    {'Month': 11, 'Day': 11, 'DOW': None, 'WOM': None},  # Veterans Day
    {'Month': 10, 'Day': 31, 'DOW': None, 'WOM': None},  # Halloween
    {'Month': 3, 'Day': 17, 'DOW': None, 'WOM': None},  # St. Patrick's Day
    {'Month': 11, 'Day': 25, 'DOW': '4', 'WOM': None},  # Black Friday
    {'Month': 12, 'Day': 26, 'DOW': None, 'WOM': None},  # Boxing Day
    ]

# Iterate over the public holidays and create the 'public_holiday' column
    df['PUBLIC_HOLIDAY'] = 0  # Initialize the column with 0 (not a public holiday)
    for holiday in public_holidays:
        month_mask = df['date'].dt.month == holiday['Month']
        day_mask = df['date'].dt.day == holiday['Day']
        dow_mask = df['date'].dt.dayofweek == int(holiday['DOW']) if holiday['DOW'] is not None else True
        wom_mask = (df['date'].dt.day - 1) // 7 + 1 == holiday['WOM'] if holiday['WOM'] is not None else True

        mask = month_mask & day_mask & dow_mask & wom_mask
        df.loc[mask, 'PUBLIC_HOLIDAY'] = 1
    return df

In [50]:
truck_manager_table = session.table('RAW_POS."TRUCK_FRANCHISE"').to_pandas()

In [51]:
algo_table = session.table('ROUTING."ALGO_DATA(With Year)"')
algo_df = algo_table.to_pandas()
df_drop=algo_df.drop("YEAR",axis=1)
final_df=df_drop.drop("SUM(ORDER_TOTAL)",axis=1)
Sales_Forecast_Training_Data_row=session.table('ANALYTICS."Sales_Forecast_Training_Data"')
Sales_Forecast_Training_Data_df = Sales_Forecast_Training_Data_row.to_pandas()
ml_df=final_df[list(Sales_Forecast_Training_Data_df.drop("Profit",axis=1).columns)]

In [52]:
df = pd.DataFrame()

df=ml_df[location_cols].drop_duplicates()

df=generate_date_data(df,datetime_object)


hours = list(range(24))
df['HOUR'] = df.apply(lambda row: hours, axis=1)
df=df.explode('HOUR', ignore_index=True)


In [53]:
trc_df = pd.DataFrame()

trc_df=ml_df[truck_cols].drop_duplicates()

trc_df=generate_date_data(trc_df,datetime_object)

In [54]:
trc_df.drop(["MONTH","DOW","DAY","WOM","YEAR","PUBLIC_HOLIDAY"],axis=1,inplace=True)
merge_df=pd.merge(df, trc_df, how='inner', on="date") 

In [55]:
wdf=session.sql("Select * from ANALYTICS.WEATHER_DATA_API")

wdf=wdf.withColumn("H",F.substring(wdf["TIME"], 12, 2).cast("integer"))

wdf=wdf.withColumn("DATE",F.substring(wdf["TIME"], 0, 10))

wdf=wdf.select("WEATHERCODE","LOCATION_ID","H","DATE" ).to_pandas()

wdf['DATE'] = pd.to_datetime(wdf['DATE'])

wdf.rename(columns = {'H':'HOUR'}, inplace = True)

weadf = pd.merge(wdf, merge_df, right_on=['LOCATION_ID','date',"HOUR"], left_on=['LOCATION_ID','DATE',"HOUR"])
weadf = weadf.drop(['date'], axis=1)
weadf = weadf.drop(['WOM'], axis=1)

In [56]:
latest_date = {'YEAR': 2022.0, 'MONTH': 10.0, 'DAY': 30.0}

# Calculate the date 1 year before the latest date
one_year_before = {'YEAR': latest_date['YEAR'] - 1, 'MONTH': latest_date['MONTH'], 'DAY': latest_date['DAY']}

# Filter the DataFrame to exclude data within the last year
holdout_df_year = algo_df[(algo_df['YEAR'] < one_year_before['YEAR']) | 
                      (algo_df['YEAR'] == one_year_before['YEAR']) & (algo_df['MONTH'] < one_year_before['MONTH']) |
                      (algo_df['YEAR'] == one_year_before['YEAR']) & (algo_df['MONTH'] == one_year_before['MONTH']) & (algo_df['DAY'] <= one_year_before['DAY'])]

X_final_scaled = holdout_df_year.copy()
#Add date column
X_final_scaled.rename(columns={"SUM(ORDER_TOTAL)": "Revenue"},inplace=True)
X_final_scaled['Date'] = pd.to_datetime(X_final_scaled[['YEAR', 'MONTH', 'DAY']])

X_final_scaled.rename(columns = {'Date':'DATE'}, inplace = True)
X_final_scaled_revenue = X_final_scaled.copy()
X_final_scaled = X_final_scaled.drop(['Revenue'], axis=1)
X_final_scaled.info()


df1_columns = set(X_final_scaled.columns)
df2_columns = set(weadf.columns)

columns_only_in_df1 = df1_columns - df2_columns
columns_only_in_df2 = df2_columns - df1_columns

print("Columns only in df1:", columns_only_in_df1)
print("Columns only in df2:", columns_only_in_df2)


merged_df = X_final_scaled.merge(weadf, on=['CITY_BOSTON_ENCODED',
 'CITY_DENVER_ENCODED',
 'CITY_New York City_encoded',
 'CITY_SEATTLE_ENCODED',
 'CITY_San Mateo_encoded',
 'DATE',
 'DAY',
 'DOW',
 'HOUR',
 'LAT',
 'LOCATION_ID',
 'LONG',
 'MENU_TYPE_BBQ_ENCODED',
 'MENU_TYPE_CHINESE_ENCODED',
 'MENU_TYPE_CREPES_ENCODED',
 'MENU_TYPE_ETHIOPIAN_ENCODED',
 'MENU_TYPE_GYROS_ENCODED',
 'MENU_TYPE_Grilled Cheese_encoded',
 'MENU_TYPE_Hot Dogs_encoded',
 'MENU_TYPE_INDIAN_ENCODED',
 'MENU_TYPE_Ice Cream_encoded',
 'MENU_TYPE_Mac & Cheese_encoded',
 'MENU_TYPE_POUTINE_ENCODED',
 'MENU_TYPE_RAMEN_ENCODED',
 'MENU_TYPE_SANDWICHES_ENCODED',
 'MENU_TYPE_TACOS_ENCODED',
 'MENU_TYPE_VEGETARIAN_ENCODED',
 'MONTH',
 'PUBLIC_HOLIDAY',
 'REGION_CA_ENCODED',
 'REGION_CO_ENCODED',
 'REGION_MA_ENCODED',
 'REGION_NY_ENCODED',
 'REGION_WA_ENCODED',
 'TRUCK_ID',
 'WEATHERCODE',
 'YEAR'], how='outer')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242811 entries, 0 to 540200
Data columns (total 39 columns):
 #   Column                                    Non-Null Count   Dtype         
---  ------                                    --------------   -----         
 0   TRUCK_ID                                  242811 non-null  int8          
 1   MONTH                                     242811 non-null  int8          
 2   HOUR                                      242811 non-null  int8          
 3   DOW                                       242811 non-null  int8          
 4   DAY                                       242811 non-null  int8          
 5   PUBLIC_HOLIDAY                            242811 non-null  int8          
 6   LAT                                       242811 non-null  float64       
 7   LONG                                      242811 non-null  float64       
 8   LOCATION_ID                               242811 non-null  int16         
 9   SUM_DAY_OF_WEEK

c:\Users\Nathan\anaconda3\envs\py38_env\lib\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


## Generating Input Data

In [57]:
working_days = 6
truck_ids = [1,2,13,14,17,21,27, 28,33,34,35, 43, 44, 46, 47] 

start_date = datetime.strptime('2021-08-23', '%Y-%m-%d')
dates = [start_date + timedelta(days=i) for i in range(working_days)]

truck_data = []


for i in range(len(truck_ids)):
    num_of_locs = random.randrange(2, 5)
    each_location_travel_distance = random.randrange(8, 12)
    max_total_travel_distance = each_location_travel_distance * num_of_locs

    truck_data.append({
        'Truck_ID': truck_ids[i],
        'Date': '2021-08-23',
        'Starting_Hour': random.randrange(8, 12),
        'Ending_Hour': random.randrange(18, 24),
        'Num_of_locs': num_of_locs,
        'each_location_travel_distance': each_location_travel_distance,
        'Max_Total_Travel_Distance': max_total_travel_distance
    })

truck_df = pd.DataFrame(truck_data)

In [58]:
session.use_schema("RAW_POS")

In [59]:
starting_locations = {}

for truck in truck_ids:
    truck_locations = X_final_scaled[X_final_scaled['TRUCK_ID'] == truck]['LOCATION_ID'].values
    starting_location = truck_locations[0] if len(truck_locations) > 0 else None
    starting_locations[truck] = starting_location

truck_df['Starting_Location'] = truck_df['Truck_ID'].map(starting_locations)

In [60]:
truck_id_to_impute = 35
imputed_location = 15098

truck_df.loc[truck_df['Truck_ID'] == truck_id_to_impute, 'Starting_Location'] = imputed_location

In [61]:
truck_df

,Truck_ID,Date,Starting_Hour,Ending_Hour,Num_of_locs,each_location_travel_distance,Max_Total_Travel_Distance,Starting_Location
0,1,2021-08-23,11,22,4,10,40,3289.0
1,2,2021-08-23,10,21,3,9,27,3289.0
2,13,2021-08-23,9,18,4,9,36,2255.0
3,14,2021-08-23,9,20,4,11,44,3289.0
4,17,2021-08-23,8,23,3,9,27,3304.0
5,21,2021-08-23,9,22,3,8,24,3304.0
6,27,2021-08-23,10,22,2,9,18,3304.0
7,28,2021-08-23,10,22,3,10,30,3304.0
8,33,2021-08-23,10,21,2,10,20,14997.0
9,34,2021-08-23,8,20,4,9,36,15098.0


In [62]:
def calculate_list_and_mod(row):
    working_hours = row['working_hour']
    num_locs = row['Num_of_locs']
    each_loc_hours = working_hours // num_locs
    remainder = working_hours % num_locs
    result_list = [each_loc_hours] * num_locs
    for i in range(remainder):
        result_list[i] += 1
    return result_list

In [8]:
def calculate_start_time(row):
    result_list=row["shift_hours"]
    num_locs = row['Num_of_locs']
    start_times = [row['Starting_Hour']]
    for i in range(1, num_locs):
        start_times.append(start_times[-1] + result_list[i - 1])

    start_times.append(row["Ending_Hour"])

    return start_times


In [64]:
truck_df["working_hour"]=truck_df['Ending_Hour']-truck_df['Starting_Hour']+1
truck_df["shift_hours"]=truck_df.apply(calculate_list_and_mod, axis=1)
truck_df["start_time"]=truck_df.apply(calculate_start_time, axis=1)

In [65]:
date = '2021-8-23'
datetime_object = datetime.strptime(date, '%Y-%m-%d')

weekadd = timedelta(days=14)

two_week=datetime_object-weekadd

sql_string='select DAY,MONTH,YEAR,TRUCK_ID,"SUM(ORDER_TOTAL)" from ROUTING."ALGO_DATA(With Year)"\
where (DAY>={} and YEAR>={} and MONTH>={}) \
and (DAY<={} and YEAR<={} and MONTH<={})'.format(two_week.day,two_week.year,two_week.month,datetime_object.day,datetime_object.year,datetime_object.month)


full_sql='select Truck_ID,SUM("SUM(ORDER_TOTAL)") as Revenue from ({}) group by TRUCK_ID  having TRUCK_ID  in (1,2,13,14,17,21,27, 28,33,34,35, 43, 44, 46, 47)'.format(sql_string)


session.sql(full_sql).show()

past_2_weeks_rows = session.sql(full_sql).collect()
past_2_weeks_df = pd.DataFrame(past_2_weeks_rows)

past_2_weeks_df_sorted = past_2_weeks_df.sort_values(by='REVENUE', ascending=False)
past_2_weeks_df_sorted

--------------------------
|"TRUCK_ID"  |"REVENUE"  |
--------------------------
|28          |669889.0   |
|33          |1047404.5  |
|47          |1974438.0  |
|46          |894930.0   |
|1           |46186.5    |
|27          |913659.0   |
|21          |414325.0   |
|13          |78465.0    |
|2           |82081.0    |
|17          |1057530.0  |
--------------------------



,TRUCK_ID,REVENUE
2,47,1974438.0
10,43,1405972.0
9,17,1057530.0
1,33,1047404.5
5,27,913659.0
3,46,894930.0
0,28,669889.0
6,21,414325.0
11,14,107709.0
8,2,82081.0


In [66]:
mean_revenue = past_2_weeks_df_sorted['REVENUE'].mean()
new_row = {'TRUCK_ID': 44, 'REVENUE': mean_revenue}
past_2_weeks_df_sorted = past_2_weeks_df_sorted.append(new_row, ignore_index=True)
new_row = {'TRUCK_ID': 34, 'REVENUE': mean_revenue}
past_2_weeks_df_sorted = past_2_weeks_df_sorted.append(new_row, ignore_index=True)
new_row = {'TRUCK_ID': 35, 'REVENUE': mean_revenue}
past_2_weeks_df_sorted = past_2_weeks_df_sorted.append(new_row, ignore_index=True)
past_2_weeks_df_sorted = past_2_weeks_df_sorted.sort_values(by='REVENUE', ascending=False)

past_2_weeks_df_sorted

C:\Users\Nathan\AppData\Local\Temp\ipykernel_475040\2885840955.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  past_2_weeks_df_sorted = past_2_weeks_df_sorted.append(new_row, ignore_index=True)
C:\Users\Nathan\AppData\Local\Temp\ipykernel_475040\2885840955.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  past_2_weeks_df_sorted = past_2_weeks_df_sorted.append(new_row, ignore_index=True)
C:\Users\Nathan\AppData\Local\Temp\ipykernel_475040\2885840955.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  past_2_weeks_df_sorted = past_2_weeks_df_sorted.append(new_row, ignore_index=True)


,TRUCK_ID,REVENUE
0,47.0,1.974438e+06
1,43.0,1.405972e+06
2,17.0,1.057530e+06
3,33.0,1.047404e+06
4,27.0,9.136590e+05
5,46.0,8.949300e+05
12,44.0,7.243824e+05
13,34.0,7.243824e+05
14,35.0,7.243824e+05
6,28.0,6.698890e+05


In [67]:
past_2_weeks_df_sorted["priority"]=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

In [68]:
past_2_weeks_df_sorted

,TRUCK_ID,REVENUE,priority
0,47.0,1.974438e+06,1
1,43.0,1.405972e+06,2
2,17.0,1.057530e+06,3
3,33.0,1.047404e+06,4
4,27.0,9.136590e+05,5
5,46.0,8.949300e+05,6
12,44.0,7.243824e+05,7
13,34.0,7.243824e+05,8
14,35.0,7.243824e+05,9
6,28.0,6.698890e+05,10


In [69]:
truck_df = pd.merge(truck_df, past_2_weeks_df_sorted, left_on=['Truck_ID'], right_on=['TRUCK_ID'])

In [70]:
truck_df

,Truck_ID,Date,Starting_Hour,Ending_Hour,Num_of_locs,each_location_travel_distance,Max_Total_Travel_Distance,Starting_Location,working_hour,shift_hours,start_time,TRUCK_ID,REVENUE,priority
0,1,2021-08-23,11,22,4,10,40,3289.0,12,"[3, 3, 3, 3]","[11, 14, 17, 20, 22]",1.0,4.618650e+04,15
1,2,2021-08-23,10,21,3,9,27,3289.0,12,"[4, 4, 4]","[10, 14, 18, 21]",2.0,8.208100e+04,13
2,13,2021-08-23,9,18,4,9,36,2255.0,10,"[3, 3, 2, 2]","[9, 12, 15, 17, 18]",13.0,7.846500e+04,14
3,14,2021-08-23,9,20,4,11,44,3289.0,12,"[3, 3, 3, 3]","[9, 12, 15, 18, 20]",14.0,1.077090e+05,12
4,17,2021-08-23,8,23,3,9,27,3304.0,16,"[6, 5, 5]","[8, 14, 19, 23]",17.0,1.057530e+06,3
5,21,2021-08-23,9,22,3,8,24,3304.0,14,"[5, 5, 4]","[9, 14, 19, 22]",21.0,4.143250e+05,11
6,27,2021-08-23,10,22,2,9,18,3304.0,13,"[7, 6]","[10, 17, 22]",27.0,9.136590e+05,5
7,28,2021-08-23,10,22,3,10,30,3304.0,13,"[5, 4, 4]","[10, 15, 19, 22]",28.0,6.698890e+05,10
8,33,2021-08-23,10,21,2,10,20,14997.0,12,"[6, 6]","[10, 16, 21]",33.0,1.047404e+06,4
9,34,2021-08-23,8,20,4,9,36,15098.0,13,"[4, 3, 3, 3]","[8, 12, 15, 18, 20]",34.0,7.243824e+05,8


## Algo

In [71]:
merged_df = merged_df.fillna({ 'SUM_PREV_YEAR_MONTH_SALES_CITY_MENU_TYPE':(merged_df['SUM_PREV_YEAR_MONTH_SALES_CITY_MENU_TYPE'].mean())})
merged_df = merged_df.fillna({ 'SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE':(merged_df['SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE'].mean())})
predicted_df = merged_df[['TRUCK_ID', 'MONTH', 'HOUR', 'DOW', 'DAY', 'PUBLIC_HOLIDAY', 'LAT',
       'LONG', 'LOCATION_ID', 'SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE',
       'SUM_PREV_YEAR_MONTH_SALES_CITY_MENU_TYPE', 'WEATHERCODE',
       'MENU_TYPE_GYROS_ENCODED', 'MENU_TYPE_CREPES_ENCODED',
       'MENU_TYPE_BBQ_ENCODED', 'MENU_TYPE_SANDWICHES_ENCODED',
       'MENU_TYPE_Mac & Cheese_encoded', 'MENU_TYPE_POUTINE_ENCODED',
       'MENU_TYPE_ETHIOPIAN_ENCODED', 'MENU_TYPE_TACOS_ENCODED',
       'MENU_TYPE_Ice Cream_encoded', 'MENU_TYPE_Hot Dogs_encoded',
       'MENU_TYPE_CHINESE_ENCODED', 'MENU_TYPE_Grilled Cheese_encoded',
       'MENU_TYPE_VEGETARIAN_ENCODED', 'MENU_TYPE_INDIAN_ENCODED',
       'MENU_TYPE_RAMEN_ENCODED', 'CITY_SEATTLE_ENCODED',
       'CITY_DENVER_ENCODED', 'CITY_San Mateo_encoded',
       'CITY_New York City_encoded', 'CITY_BOSTON_ENCODED',
       'REGION_NY_ENCODED', 'REGION_MA_ENCODED', 'REGION_CO_ENCODED',
       'REGION_WA_ENCODED', 'REGION_CA_ENCODED']]
predicted_df

,TRUCK_ID,MONTH,HOUR,DOW,DAY,PUBLIC_HOLIDAY,LAT,LONG,LOCATION_ID,SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE,...,CITY_SEATTLE_ENCODED,CITY_DENVER_ENCODED,CITY_San Mateo_encoded,CITY_New York City_encoded,CITY_BOSTON_ENCODED,REGION_NY_ENCODED,REGION_MA_ENCODED,REGION_CO_ENCODED,REGION_WA_ENCODED,REGION_CA_ENCODED
0,17,9,16.0,4,10,0,39.748742,-104.972158,3304,-0.276074,...,0,1,0,0,0,0,0,1,0,0
1,17,9,17.0,4,10,0,39.748742,-104.972158,3304,-0.276074,...,0,1,0,0,0,0,0,1,0,0
2,17,9,18.0,4,10,0,39.748742,-104.972158,3304,-0.276074,...,0,1,0,0,0,0,0,1,0,0
3,17,9,19.0,4,10,0,39.748742,-104.972158,3304,-0.276074,...,0,1,0,0,0,0,0,1,0,0
4,17,9,20.0,4,10,0,39.748742,-104.972158,3304,-0.276074,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3625006,3,8,23,0,23,0,42.356017,-71.050726,4136,-0.080459,...,0,0,0,0,1,0,1,0,0,0
3625007,13,8,23,0,23,0,42.356017,-71.050726,4136,-0.080459,...,0,0,0,0,1,0,1,0,0,0
3625008,10,8,23,0,23,0,42.356017,-71.050726,4136,-0.080459,...,0,0,0,0,1,0,1,0,0,0
3625009,5,8,23,0,23,0,42.356017,-71.050726,4136,-0.080459,...,0,0,0,0,1,0,1,0,0,0


In [72]:
model = joblib.load(open('model.joblib',"rb"))



[15:29:47] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



In [73]:
predicted_df["HOUR"]=predicted_df["HOUR"].astype(int)
model.predict(predicted_df)

C:\Users\Nathan\AppData\Local\Temp\ipykernel_475040\1277206958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted_df["HOUR"]=predicted_df["HOUR"].astype(int)


array([3011.4856 , 5846.3574 , 7421.3857 , ...,  112.86045,  276.80035,
        215.89268], dtype=float32)

In [74]:
predictions = model.predict(predicted_df)

In [75]:
predicted_df["predictions"]=predictions

C:\Users\Nathan\AppData\Local\Temp\ipykernel_475040\3702424682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted_df["predictions"]=predictions


In [76]:
unique_location_ids = predicted_df['LOCATION_ID'].unique()

hourly_location_df = pd.DataFrame(columns=[str(hour) for hour in range(1, 25)])

for location_id in unique_location_ids:
    row_data = {str(hour): 0 for hour in range(1, 25)}
    hourly_location_df = hourly_location_df.append(row_data, ignore_index=True)

hourly_location_df['LOCATION_ID'] = unique_location_ids
hourly_location_df = hourly_location_df[['LOCATION_ID'] + [str(hour) for hour in range(1, 25)]]

C:\Users\Nathan\AppData\Local\Temp\ipykernel_475040\2348948760.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hourly_location_df = hourly_location_df.append(row_data, ignore_index=True)
C:\Users\Nathan\AppData\Local\Temp\ipykernel_475040\2348948760.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hourly_location_df = hourly_location_df.append(row_data, ignore_index=True)
C:\Users\Nathan\AppData\Local\Temp\ipykernel_475040\2348948760.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hourly_location_df = hourly_location_df.append(row_data, ignore_index=True)
C:\Users\Nathan\AppData\Local\Temp\ipykernel_475040\2348948760.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a f

In [77]:
route_df=predicted_df[["TRUCK_ID","LOCATION_ID","LAT","LONG","predictions","HOUR"]]

In [78]:
truck_df["current_location"]=None
truck_df["next_start_time"]=None
truck_df["location_visited"]=pd.Series([[]] * len(truck_df))
truck_df["predicted_earning"]=pd.Series([[]] * len(truck_df))

In [79]:
def calculate_next_start_time(start_times, current_hour):
    if current_hour not in start_times:
        return None
    index = start_times.index(current_hour) + 1
    if index == len(start_times):
        return None
    return start_times[index]


In [80]:
def haversine_distance(lat1, lon1, lat2, lon2):
        # Convert latitude and longitude from degrees to radians
        lat1_rad = math.radians(lat1)
        lon1_rad = math.radians(lon1)
        lat2_rad = math.radians(lat2)
        lon2_rad = math.radians(lon2)

        # Haversine formula
        dlon = lon2_rad - lon1_rad
        dlat = lat2_rad - lat1_rad
        a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        distance = 6371 * c  # Radius of the Earth in kilometers
        return distance

In [102]:
def filter_list(current_loc,available_locations, distance_param):
  
    current_lat=route_df[route_df["LOCATION_ID"]==current_loc]["LAT"].values[0]
    
    current_long=route_df[route_df["LOCATION_ID"]==current_loc]["LONG"].values[0]
    
    new_list=[]
    for i in available_locations:
        lat=route_df[route_df["LOCATION_ID"]==i]["LAT"].values[0]
        long=route_df[route_df["LOCATION_ID"]==i]["LONG"].values[0]
       
        distance=haversine_distance(current_lat, current_long, lat, long)
       
        if distance_param >= distance:
           
            new_list.append(i)
    return new_list                        

In [82]:
truck_df_copy = truck_df.copy()

In [104]:
truck_df["current_location"]=truck_df["Starting_Location"]

In [105]:
# Iterate over each hour
for hour in range(24):
    
    trucks_starting_now = truck_df[truck_df['start_time'].apply(lambda start_times: hour in start_times)]
    trucks_starting_now.sort_values(by='priority',ascending=True).reset_index(drop=True)
   
    
    trucks_starting_now ['next_start_time'] =  trucks_starting_now ['start_time'].apply(lambda start_times: calculate_next_start_time(start_times,hour))
  
 
    
    
    
    available_locations = []
    for index, truck in trucks_starting_now.iterrows():
     
     
        current_hour = str(hour)
        
        
        for loc_index, loc_row in  hourly_location_df.iterrows():
            if loc_row[current_hour]==0:
                available_locations.append(loc_row["LOCATION_ID"])
            else:
                if loc_row[current_hour]==truck["Truck_ID"]:
                    available_locations.append(loc_row["LOCATION_ID"])
                    
                    hourly_location_df.loc[hourly_location_df[current_hour]==truck["Truck_ID"],current_hour]=0
                    
    available_locations=list(dict.fromkeys(available_locations))
    
    
    for index, truck in trucks_starting_now.iterrows():
         
         
         ### ditance calculator
         ## remove values from available_locatio based on distance
         ##if u wan any other way also can
        new_list=[]
        if pd.notna(truck["current_location"]) :
            new_list=filter_list(truck["current_location"], available_locations, truck['each_location_travel_distance'])
        
        if pd.notna(truck['next_start_time']):
            route_df_h=route_df[(route_df["HOUR"]>=hour) & (route_df["HOUR"]<=  int(truck['next_start_time']))] 
            
            route_df_truck=route_df_h[route_df_h["TRUCK_ID"]==truck["Truck_ID"]]
            route_df_loc=route_df_truck[route_df_truck["LOCATION_ID"].isin(new_list)]
            
            if route_df_loc.size>0:
                    
                route_df_grpby=route_df_loc.groupby("LOCATION_ID").sum()["predictions"]
            
                location =  route_df_grpby.idxmax()
        
            
                truck_df.at[index,'current_location'] = location
                truck_df.at[index,'location_visited'] =   list(truck_df.at[index,'location_visited']) + [location]
                truck_df.at[index,'predicted_earning'] = list(truck_df.at[index,'predicted_earning']) + [route_df_grpby.max()]
                

                available_locations.remove(location)

            if pd.notna(truck['next_start_time']):
            
                for i in range(hour,int(truck['next_start_time'])):
        
                    column=str(i)
                    hourly_location_df.loc[hourly_location_df["LOCATION_ID"]==location,str(i)]=truck["Truck_ID"]  

C:\Users\Nathan\AppData\Local\Temp\ipykernel_475040\1456191818.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trucks_starting_now ['next_start_time'] =  trucks_starting_now ['start_time'].apply(lambda start_times: calculate_next_start_time(start_times,hour))
C:\Users\Nathan\AppData\Local\Temp\ipykernel_475040\1456191818.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trucks_starting_now ['next_start_time'] =  trucks_starting_now ['start_time'].apply(lambda start_times: calculate_next_start_time(sta

In [106]:
truck["location_visited"]

[5165, 1805]

In [107]:
truck_df.head()

,Truck_ID,Date,Starting_Hour,Ending_Hour,Num_of_locs,each_location_travel_distance,Max_Total_Travel_Distance,Starting_Location,working_hour,shift_hours,start_time,TRUCK_ID,REVENUE,priority,current_location,next_start_time,location_visited,predicted_earning
0,1,2021-08-23,11,22,4,10,40,3289.0,12,"[3, 3, 3, 3]","[11, 14, 17, 20, 22]",1.0,46186.5,15,3384.0,None,"[2314, 3195, 3384, 3384]","[21662.208984375, 12465.9833984375, 32482.3613..."
1,2,2021-08-23,10,21,3,9,27,3289.0,12,"[4, 4, 4]","[10, 14, 18, 21]",2.0,82081.0,13,1796.0,None,"[2269, 1796, 1796]","[38517.40234375, 32610.1953125, 50182.0859375]"
2,13,2021-08-23,9,18,4,9,36,2255.0,10,"[3, 3, 2, 2]","[9, 12, 15, 17, 18]",13.0,78465.0,14,3632.0,None,"[2998, 2998, 3632, 3632]","[23408.6875, 15517.5302734375, 9179.876953125,..."
3,14,2021-08-23,9,20,4,11,44,3289.0,12,"[3, 3, 3, 3]","[9, 12, 15, 18, 20]",14.0,107709.0,12,3488.0,None,"[2900, 2900, 3488, 3488]","[39449.875, 25573.009765625, 23928.201171875, ..."
4,17,2021-08-23,8,23,3,9,27,3304.0,16,"[6, 5, 5]","[8, 14, 19, 23]",17.0,1057530.0,3,1730.0,None,"[1233, 1730, 1730]","[248674.71875, 169996.671875, 151586.09375]"


In [124]:
def get_lat_long(location_id):
    row = algo_df[algo_df['LOCATION_ID'] == location_id]
    if not row.empty:
        return row['LAT'].values[0], row['LONG'].values[0]
    return None, None

# Calculate the total distance traveled for each truck
total_distances = []
for _, row in truck_df.iterrows():
    starting_location = row['Starting_Location']
    location_visited = row['location_visited']
    
    total_distance = 0
    prev_lat, prev_lon = get_lat_long(starting_location)  # Get the latitude and longitude of the starting location
    
    # Loop through each location in location_visited
    for location_id in location_visited:
        lat, lon = get_lat_long(location_id)
        
        # If latitude and longitude are found, calculate the distance between the two locations
        if lat is not None and lon is not None:
            total_distance += haversine_distance(prev_lat, prev_lon, lat, lon)
        
            # Update the previous location
            prev_lat, prev_lon = lat, lon
    
    total_distances.append(total_distance)

# Add the total_distances list as a new column to the truck_df
truck_df['total_distance_traveled'] = total_distances

In [125]:
truck_df

,Truck_ID,Date,Starting_Hour,Ending_Hour,Num_of_locs,each_location_travel_distance,Max_Total_Travel_Distance,Starting_Location,working_hour,shift_hours,start_time,TRUCK_ID,REVENUE,priority,current_location,next_start_time,location_visited,predicted_earning,total_distance_traveled
0,1,2021-08-23,11,22,4,10,40,3289.0,12,"[3, 3, 3, 3]","[11, 14, 17, 20, 22]",1.0,4.618650e+04,15,3384.0,None,"[2314, 3195, 3384, 3384]","[21662.208984375, 12465.9833984375, 32482.3613...",8.384929
1,2,2021-08-23,10,21,3,9,27,3289.0,12,"[4, 4, 4]","[10, 14, 18, 21]",2.0,8.208100e+04,13,1796.0,None,"[2269, 1796, 1796]","[38517.40234375, 32610.1953125, 50182.0859375]",4.727919
2,13,2021-08-23,9,18,4,9,36,2255.0,10,"[3, 3, 2, 2]","[9, 12, 15, 17, 18]",13.0,7.846500e+04,14,3632.0,None,"[2998, 2998, 3632, 3632]","[23408.6875, 15517.5302734375, 9179.876953125,...",9.319448
3,14,2021-08-23,9,20,4,11,44,3289.0,12,"[3, 3, 3, 3]","[9, 12, 15, 18, 20]",14.0,1.077090e+05,12,3488.0,None,"[2900, 2900, 3488, 3488]","[39449.875, 25573.009765625, 23928.201171875, ...",3.736695
4,17,2021-08-23,8,23,3,9,27,3304.0,16,"[6, 5, 5]","[8, 14, 19, 23]",17.0,1.057530e+06,3,1730.0,None,"[1233, 1730, 1730]","[248674.71875, 169996.671875, 151586.09375]",15.177074
5,21,2021-08-23,9,22,3,8,24,3304.0,14,"[5, 5, 4]","[9, 14, 19, 22]",21.0,4.143250e+05,11,14800.0,None,"[1550, 1321, 14800]","[100875.8828125, 81790.90625, 82035.03125]",12.775782
6,27,2021-08-23,10,22,2,9,18,3304.0,13,"[7, 6]","[10, 17, 22]",27.0,9.136590e+05,5,15501.0,None,"[1056, 15501]","[136497.921875, 196333.09375]",7.225401
7,28,2021-08-23,10,22,3,10,30,3304.0,13,"[5, 4, 4]","[10, 15, 19, 22]",28.0,6.698890e+05,10,1356.0,None,"[15457, 1356, 1356]","[74791.4609375, 84143.0390625, 75035.8984375]",5.437979
8,33,2021-08-23,10,21,2,10,20,14997.0,12,"[6, 6]","[10, 16, 21]",33.0,1.047404e+06,4,15465.0,None,"[15019, 15465]","[144473.765625, 210992.921875]",8.345571
9,34,2021-08-23,8,20,4,9,36,15098.0,13,"[4, 3, 3, 3]","[8, 12, 15, 18, 20]",34.0,7.243824e+05,8,15464.0,None,"[5176, 5176, 1808, 15464]","[87284.4921875, 52510.99609375, 31896.60742187...",15.025554


In [126]:
hourly_location_df[hourly_location_df["9"]!=0]

,LOCATION_ID,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
243,3439,0,0,0,0,0,0,0,44,44,...,0,0,0,0,0,0,0,0,0,0
297,1233,0,0,0,0,0,0,0,17,17,...,0,0,0,0,0,0,0,0,0,0
356,2900,0,0,0,0,0,0,0,0,14,...,0,0,0,0,0,0,0,0,0,0
453,1030,0,0,0,0,0,0,0,0,17,...,0,0,0,0,0,0,0,0,0,0
556,1550,0,0,0,0,0,0,0,0,21,...,0,0,0,0,0,0,0,0,0,0
1039,5150,0,0,0,0,0,0,0,0,34,...,0,0,0,0,0,0,0,0,0,0
1146,5176,0,0,0,0,0,0,0,34,34,...,35,35,35,35,35,35,1,0,0,0
1185,10398,0,0,0,0,0,0,0,0,44,...,44,0,0,0,0,0,0,0,0,0
1690,2998,0,0,0,0,0,0,0,0,13,...,0,0,0,0,0,0,0,0,0,0


In [127]:
truck_manager_table['Name'] = truck_manager_table['FIRST_NAME'] +' '+ truck_manager_table['LAST_NAME']

In [128]:
truck_manager_table.head()

,FRANCHISE_ID,FIRST_NAME,LAST_NAME,FRANCHISE_CITY,FRANCHISE_COUNTRY,E_MAIL,PHONE_NUMBER,TRUCK_ID,MENU_TYPE_ID,TRUCK_PRIMARY_CITY,...,ISO_REGION,TRUCK_COUNTRY,ISO_COUNTRY_CODE,FRANCHISE_FLAG,YEAR,MAKE,MODEL,EV_FLAG,TRUCK_OPENING_DATE,Name
0,1,Corporate,Owned,San Mateo,United States,TastyBytes@Gmail.com,434-928-2233,1,1,San Mateo,...,CA,United States,US,0,2009,Ford_,Step Van,0,2019-01-01,Corporate Owned
1,1,Corporate,Owned,San Mateo,United States,TastyBytes@Gmail.com,434-928-2233,2,2,San Mateo,...,CA,United States,US,0,2015,Ford_,Step Van,0,2020-07-01,Corporate Owned
2,2,Annette,Gates,San Mateo,United States,Annette.Gates@yahoo.com,949-148-5165,3,3,San Mateo,...,CA,United States,US,1,2004,Freightliner,MT45 Utilimaster,0,2021-10-01,Annette Gates
3,3,Steven,Phelps,San Mateo,United States,Steven.Phelps@aol.com,746-288-2047,4,4,San Mateo,...,CA,United States,US,1,1997,Chevrolet,P30,1,2021-01-01,Steven Phelps
4,4,Sandra,Dunn,San Mateo,United States,Sandra.Dunn@ymail.com,043-798-0237,5,5,San Mateo,...,CA,United States,US,1,2010,Custom,Van,1,2022-04-01,Sandra Dunn


In [129]:
selected_truck_ids = [1,2,13,14,17,21,27,28,33,34,35,43,44,46,47] 

In [130]:
algo_df

,TRUCK_ID,MONTH,HOUR,DOW,DAY,PUBLIC_HOLIDAY,SUM(ORDER_TOTAL),LAT,LONG,LOCATION_ID,...,CITY_SEATTLE_ENCODED,CITY_DENVER_ENCODED,CITY_San Mateo_encoded,CITY_New York City_encoded,CITY_BOSTON_ENCODED,REGION_NY_ENCODED,REGION_MA_ENCODED,REGION_CO_ENCODED,REGION_WA_ENCODED,REGION_CA_ENCODED
0,17,9,16,4,10,0,1305.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
1,17,9,17,4,10,0,610.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
2,17,9,18,4,10,0,3461.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
3,17,9,19,4,10,0,5221.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
4,17,9,20,4,10,0,8929.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540196,21,6,18,4,4,0,3517.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
540197,21,6,19,4,4,0,2466.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
540198,21,6,20,4,4,0,2297.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0
540199,21,6,21,4,4,0,2289.0,39.748742,-104.972158,3304,...,0,1,0,0,0,0,0,1,0,0


In [136]:
filtered_manager_table = truck_manager_table[truck_manager_table['TRUCK_ID'].isin(selected_truck_ids)].copy()

# Merge the 'truck_manager_table' DataFrame to get the names of truck managers
truck_manager_merged_df = truck_df.merge(filtered_manager_table[['TRUCK_ID', 'Name', 'TRUCK_PRIMARY_CITY']], on='TRUCK_ID', how='outer')

# Drop the unnecessary columns
# truck_manager_merged_df.drop(columns=['FIRST_NAME', 'LAST_NAME'], inplace=True)

In [137]:
truck_manager_merged_df = truck_manager_merged_df.drop_duplicates('Truck_ID')

In [138]:
truck_manager_merged_df = truck_manager_merged_df.reset_index(drop=True)

In [141]:
truck_manager_merged_df = truck_manager_merged_df.rename(columns={'TRUCK_PRIMARY_CITY': 'City'})

In [142]:
truck_manager_merged_df.to_csv("truck_manager_merged_df.csv")

In [143]:
truck_manager_merged_df

,Truck_ID,Date,Starting_Hour,Ending_Hour,Num_of_locs,each_location_travel_distance,Max_Total_Travel_Distance,Starting_Location,working_hour,shift_hours,...,TRUCK_ID,REVENUE,priority,current_location,next_start_time,location_visited,predicted_earning,total_distance_traveled,Name,City
0,1,2021-08-23,11,22,4,10,40,3289.0,12,"[3, 3, 3, 3]",...,1.0,4.618650e+04,15,3384.0,None,"[2314, 3195, 3384, 3384]","[21662.208984375, 12465.9833984375, 32482.3613...",8.384929,Corporate Owned,San Mateo
1,2,2021-08-23,10,21,3,9,27,3289.0,12,"[4, 4, 4]",...,2.0,8.208100e+04,13,1796.0,None,"[2269, 1796, 1796]","[38517.40234375, 32610.1953125, 50182.0859375]",4.727919,Corporate Owned,San Mateo
2,13,2021-08-23,9,18,4,9,36,2255.0,10,"[3, 3, 2, 2]",...,13.0,7.846500e+04,14,3632.0,None,"[2998, 2998, 3632, 3632]","[23408.6875, 15517.5302734375, 9179.876953125,...",9.319448,Jacob Nguyen,San Mateo
3,14,2021-08-23,9,20,4,11,44,3289.0,12,"[3, 3, 3, 3]",...,14.0,1.077090e+05,12,3488.0,None,"[2900, 2900, 3488, 3488]","[39449.875, 25573.009765625, 23928.201171875, ...",3.736695,Jacob Nguyen,San Mateo
4,17,2021-08-23,8,23,3,9,27,3304.0,16,"[6, 5, 5]",...,17.0,1.057530e+06,3,1730.0,None,"[1233, 1730, 1730]","[248674.71875, 169996.671875, 151586.09375]",15.177074,Corporate Owned,Denver
5,21,2021-08-23,9,22,3,8,24,3304.0,14,"[5, 5, 4]",...,21.0,4.143250e+05,11,14800.0,None,"[1550, 1321, 14800]","[100875.8828125, 81790.90625, 82035.03125]",12.775782,Corporate Owned,Denver
6,27,2021-08-23,10,22,2,9,18,3304.0,13,"[7, 6]",...,27.0,9.136590e+05,5,15501.0,None,"[1056, 15501]","[136497.921875, 196333.09375]",7.225401,David Miller,Denver
7,28,2021-08-23,10,22,3,10,30,3304.0,13,"[5, 4, 4]",...,28.0,6.698890e+05,10,1356.0,None,"[15457, 1356, 1356]","[74791.4609375, 84143.0390625, 75035.8984375]",5.437979,David Miller,Denver
8,33,2021-08-23,10,21,2,10,20,14997.0,12,"[6, 6]",...,33.0,1.047404e+06,4,15465.0,None,"[15019, 15465]","[144473.765625, 210992.921875]",8.345571,Corporate Owned,Seattle
9,34,2021-08-23,8,20,4,9,36,15098.0,13,"[4, 3, 3, 3]",...,34.0,7.243824e+05,8,15464.0,None,"[5176, 5176, 1808, 15464]","[87284.4921875, 52510.99609375, 31896.60742187...",15.025554,Kevin Cisneros,Seattle
